# Mundipharma

## Beachten im nächsten Jahr
* **Seitenenden werden nicht erkannt. Einige Adresse manuell einfügen!**
* Totale wurden manuell berechnet
* Letzten drei Zeilen entfernt
* Columnen und Area mussten definiert werden

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [3]:
# Read pdf into DataFrame

columns = [21.2, 48.1, 224.32, 295.15, 339.5, 472.15, 507.72, 548.98, 594.16, 634.69, 676.11, 718.01, 758.76, 780.81]
#area = [28.9559518348868, 12.820151963112876, 581.465980673998, 825.2729943703393]

#df = tabula.read_pdf("Zuwendungen_an_medizinische_Fachkreise_2018.pdf", pages='all', lattice=True, columns=columns, guess=False, area=area)
df = tabula.read_pdf("Zuwendungen-an-medizinische-Fachkreise-2019b.pdf", pages='all', lattice=True, columns=columns, guess=False)

## Format Table

In [24]:
#Rename Columns
df_export = df.copy()
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1', 'empty2']


#Shift all where 'name' is empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Copy empty2 to empty1 if empty1 empty
df_export.loc[df_export['total'].isna(), 'total'] = df_export.empty1

#Type by text
index_hco = df_export[df_export['name'].str.contains("INDIVIDUAL NAMED DISCLOSURE", na=False)].index[1]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')



#Remove empty rows
df_export = df_export.dropna(subset=['location'], how='all')
df_export = df_export[~df_export.name.str.contains('Full Name')]
df_export = df_export[0: -3]

#Remove empties
df_export.drop(columns=['empty1', 'empty2'], inplace=True)

#Remove HCPs in Text
df_export = df_export.replace("HCPs", '', regex=False)


write_to_excel(df_export, 'tmp.xlsx', open=True)

#Cleanup Numbers and sum
df_export = remove_in_numbers(df_export, "'")
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add empty fields
df_export = add_plz(df_export)

#Remove Carination
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Revert Name
df_export = revert_name(df_export, " ")

export_list(df_export, 'mundipharma')

ValueError: Unable to parse string "HCPs278" at position 63

In [12]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)